In [ ]:
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

import json
import numpy as np
import logging

from data.tokenizer import (
    AudioTokenizer,
    tokenize_audio,
)
from vocos import Vocos
from data.collation import get_text_token_collater
from models.vallex import VALLE
from utils.g2p import PhonemeBpeTokenizer
from descriptions import *
from macros import *
from examples import *
import torch

import langid
langid.set_languages(['en', 'zh', 'ja'])

device = torch.device('cuda', 0)

text_tokenizer = PhonemeBpeTokenizer(tokenizer_path='./utils/g2p/bpe_69.json')
text_collater = get_text_token_collater()

# Vocos decoder
vocos = Vocos.from_pretrained('charactr/vocos-encodec-24khz').to(device)


model = VALLE(
        N_DIM,
        NUM_HEAD,
        NUM_LAYERS,
        norm_first=True,
        add_prenet=False,
        prefix_mode=PREFIX_MODE,
        share_embedding=True,
        nar_scale_factor=1.0,
        prepend_bos=True,
        num_quantizers=NUM_QUANTIZERS,
    )
checkpoint = torch.load('./checkpoints/vallex-checkpoint.pt', map_location='cpu')
missing_keys, unexpected_keys = model.load_state_dict(
    checkpoint['model'], strict=True
)
assert not missing_keys
model.eval()

# 言語について
|言語|lang_token / accent|
|-------|----|
|English|`[EN]`|
|中文|`[ZH]`|
|日本語|`[JA]`|
|Mix|` `|

※ accentは `no-accent` が存在します

# 音素について
※GPTから持ってきただけなので適当です。
参考程度に

| 記号                   | 説明                                                                                      |
|------------------------|-------------------------------------------------------------------------------------------|
| ! (エクスクラメーションマーク): 5 | 強調や驚き、興奮を表現するために使用されるエクスクラメーションマーク。                       |
| # (ハッシュ): 6              | ハッシュマークとしてソーシャルメディアなどで使用される。                                       |
| * (アスタリスク): 7         | 強調、注釈などに使用されるアスタリスク。                                                  |
| , (カンマ): 8               | 文を区切り、要素を区別するために使用されるカンマ。                                          |
| - (ハイフン): 9              | 単語の分割や合成、範囲を示すために使用されるハイフン。                                        |
| . (ピリオド): 10             | 文の終了を示すピリオドまたはドット。                                                      |
| = (等号): 11                | 数学的な表現に使用される等号。                                                            |
| ? (疑問符): 12              | 疑問文を示す疑問符。                                                                       |
| N: 13                     | 大文字の「N」。                                                                           |
| Q: 14                     | 大文字の「Q」。                                                                           |
| ^: 15                     | キャレット。異なる文脈で指数や挿入を示すために使用される。                                     |
| _ (アンダースコア): 16        | 単語の区切りやスペースを示すアンダースコア。                                                 |
| ` (バッククォート): 17        | プログラミングなどで特殊な意味を持つバッククォートまたはバックチルダ。                             |
| a, b, d, e, ... z: 18-40  | 英小文字のアルファベット。                                                                 |
| ~: 41                     | チルダ。数学やプログラミングなどで異なる文脈で使用される。                                     |
| æ: 42                    | ラテン語アルファベットの一部で、北欧言語で使用される特殊な文字。                                 |
| ç: 43                    | ラテン語アルファベットの一部で、ポルトガル語やトルコ語などで使用される特殊な文字。                  |
| ð: 44                    | イセランダイグラフ。英語やアイスランド語などで使用される特定の子音。                             |
| ŋ: 45                    | エングマ。英語で特定の音の表現に使用される文字。                                              |
| ɑ, ɔ, ə, ɛ, ... ʒ: 46-60 | 国際音声記号（IPA）における様々な音を表す記号。                                               |
| ʰ: 61                     | スーパースクリプト「ʰ」。子音の後ろに来て、音の発声を示すために使用されることがある。                  |
| ˈ, ˌ: 62-63               | プライムとコンマ。音声学で使用され、強勢や発音の区別に関連している。                              |
| θ: 64                     | シータ。音声学や言語学で特定の音を表す子音。                                                  |
| …: 65                     | ホリゾンタルエリプシス。省略を示す。                                                      |


In [45]:
text = 'こんにちは、これは合成音声の読み上げです。'
phoneme_text = 'ko↓nn^itʃiwa~,_ko↑ɾewa_go↑oseeo↓Nseeno_jo↑miage↓desɯ*.'
from_phoneme = True # これをTrueにすると音素から合成音声を生成する

lang_token = '[JA]'
accent = '[JA]'

prompt_path = './prompts/hoshino2.npz' # プロンプトのpath



lang = token2lang[lang_token]
text = lang_token + text + lang_token

# load prompt
prompt_data = np.load(prompt_path)
audio_prompts = prompt_data['audio_tokens']
text_prompts = prompt_data['text_tokens']
lang_pr = prompt_data['lang_code']
lang_pr = code2lang[int(lang_pr)]

# numpy to tensor
audio_prompts = torch.tensor(audio_prompts).type(torch.int32).to(device)
text_prompts = torch.tensor(text_prompts).type(torch.int32)

enroll_x_lens = text_prompts.shape[-1]

# tokenizeで音素に分ける
phone_tokens, langs = text_tokenizer.tokenize(text=f'_{text}'.strip())

# 音素の参照先はここ
# ./utils/g2p/bpe_69.json
with open('./utils/g2p/bpe_69.json', 'r', encoding='utf-8') as f:
    bpe_69 = json.load(f)
vocab_list = bpe_69['model']['vocab']
reverse_vocab_list = {value: key for key, value in vocab_list.items()}

if from_phoneme:
    phone_tokens = [vocab_list[str] for str in list(phoneme_text)]
    print('入力音素: ' + phoneme_text)
    print('出力音素番号: ' + ', '.join(str(vale) for vale in phone_tokens))
else:
    phoneme_list = [reverse_vocab_list[num] for num in phone_tokens]
    # 音素トークンを音素へ置き換えた物を表示
    print('入力テキスト: ' + text)
    print('出力音素: ' + ''.join(phoneme_list))



text_tokens, text_tokens_lens = text_collater(
    [
        phone_tokens
    ]
)
text_tokens = torch.cat([text_prompts, text_tokens], dim=-1)
text_tokens_lens += enroll_x_lens

入力音素: ko↓nn^itʃiwa~,_ko↑ɾewa_go↑oseeo↓Nseeno_jo↑miage↓desɯ*.
出力音素番号: 27, 31, 69, 30, 30, 15, 25, 34, 57, 25, 37, 18, 41, 8, 16, 27, 31, 67, 56, 21, 37, 18, 16, 23, 31, 67, 31, 33, 21, 21, 31, 69, 13, 33, 21, 21, 30, 31, 16, 26, 31, 67, 29, 25, 18, 23, 21, 69, 20, 21, 33, 53, 7, 10


In [46]:
with torch.no_grad():
    model.to(device)

    # accent control
    encoded_frames = model.inference(
        text_tokens.to(device),
        text_tokens_lens.to(device),
        audio_prompts,
        enroll_x_lens=enroll_x_lens,
        top_k=-100,
        temperature=1,
        prompt_language=lang_pr,
        text_language=langs if accent == 'no-accent' else lang,
        best_of=5,
    )
    # Decode with Vocos
    frames = encoded_frames.permute(2,0,1)
    features = vocos.codes_to_features(frames)
    samples = vocos.decode(features, bandwidth_id=torch.tensor([2], device=device))

    # model.to('cpu')
    model.to('cuda:0')
    torch.cuda.empty_cache()

message = f'sythesized text: {text}'
[ audio_SAMPLE_RATE, audio_array ] = (24000, samples.squeeze(0).cpu().numpy())

# 結果の表示
Audio(audio_array, rate=audio_SAMPLE_RATE)
# write_wav("test.wav", audio_SAMPLE_RATE, audio_array)

VALL-E EOS [557 -> 934]
